# Домашнее задание №6. Ансамбли: случайный лес, градиентный бустинг.

## Ф.И.О: _________

### Описание.

Домашнее задание состоит из 2-х частей:
  - реализация модулей:
    - bagger
    - booster
    - sampler
-------------------
 * Реализации можно частично проверить через юнит-тесты (запускаются командой ``pytest tests``).
-------------------
  - экспериментальная часть

На проверку требуется отправить zip архив, который будет содержать следующие файлы:
  - модуль ``ensemble`` с реализованными модулями(bagger, booster, sampler)
  - заполненный блокнот (эксперименты) в формате .ipynb 
  - заполненный блокнот в формате .html (File -> Save and Export Notebook As -> HTML -> ...)

## 1.1 Сэмплирование случайных объектов и признаков (2 points)

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

`BaseSampler` класс, который будет упрощать семплирование различных подмассивов данных. 
Реализуйте метод:
  - `sample_indices`, который по числу сущностей `n_objects` возращает случайную подвыборку индексов.

Используйте атрибуты:
  - `self.random_state`, чтобы результаты семпплирования воспроиводились
  - `self.bootstrap`, если нужно выбрать случайную подвыборку с возвращением.

У класса `ObjectSampler` реализован метод:
  - `sample`, который возвращает случайную подвыборку объектов обучения и ответы для них.

В классе `FeaturesSampler` реализован метод:
- `sample`, который возвращает случайную подвыборку признаков для объектов.

## 1.2. Бэггинг (5 points)

Суть бэггинга заключается в обучении нескольких "слабых" базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Вам предлагается реализовать несколько методов класса `Bagger`:
* `fit` - обучение базовых моделей
* `predict_proba` - вычисление вероятностей ответов.

В данном задании реализация **случайного леса будет бэггингом над решающими деревьями**. 

Реализация случайного леса представлена в классе `RandomForestClassifier`.

## 1.3. Градиентный бустинг (7 points)

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. 

Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. У каждого следующего алгоритма вычисляется ___вес___ $\omega$, с которым он входит в ансамбль. 

Также есть параметр скорости обучения (___learning rate___ - $lr$), который не позволяет алгоритму переобучитсья. 

Вес $\omega$ можно находить, используя одномерную оптимизацию. 

Рассмотрим процедуру обучения по шагам (будем рассматривать случай бинарной классификации c метками классов {0,1}, чтобы не усложнять жизнь):
1. Настройка базового алгоритма $b_0$ (в данном случае это решающее дерево).
    
    $$\text{Алгоритм настраиваются на $y$ с помощью функции MSE.}$$
    
2. Будем обозначать текущий небазовый алгоритм - $a$:
    
    $$a_i(x) = \sum_{j=0}^i \omega_j b_j(x) $$
    
3. Настройка базового алгоритма $b_i$ (обычно это регрессионное дерево):
    
    $$b_i = \arg \min_b \sum_{j=1}^l (b(x_j) + \nabla L(a_{i-1}(x_j), y))^2,$$
    т.е. выход очередного базового алгоритма подстраивается под антиградиент функции потерь
    
4. Настройка веса базового алгоритма $\omega_i$:
    
    $$\omega_i = \min_{\omega > 0} \sum_{j=1}^l L(a_{i-1} + \omega b_i(x_j), y) $$
    
Для задачи классфикации будем использовать логистическую функцию потерь. Немного упростим ее:

$$L = -y\log\sigma(a) - (1-y)\log(1 - \sigma(a)) = -\log(1 - \sigma(a)) - y \log \frac{\sigma(a)}{1 - \sigma(a)},$$
где $\sigma$ - функция сигмоиды. 

Ответ после очередного базового алгоритма надо прогонять через сигмоиду, т.к. не гарантируется, что ответы будут лежать на [0,1] - в этом особенность базового алгоритма (который является регрессионным).

Преобразуем:
$$\log (1 - \sigma(a)) = \log \frac{1}{1 + \exp(a)} = -\log(1 + \exp(a)) $$

$$\log (\frac{\sigma(a)}{1 - \sigma(a)}) = \log(\exp(a)) = a $$
 
Таким образом:

$$L = -ya + \log(1 + \exp(a))$$

Тогда будем вычислять градиент как:
 
$$\nabla L = - y + \sigma(a)$$

В классе `Booster` реализуйте методы:
* `_fit_first_estimator` – построение первой модели (первого приближения данных);
* `fit` – обучение алгоритма градиентного бустинга (обучение первой и последующих базовых моделей);
* `predict` – получение предсказаний алгоритма градиентного бустинга.

В классе `GradientBoostingClassifier` реализуйте методы:
* `_fit_base_estimator` - обучение очередной базовой модели;
* `_gradient` - расчет градиента функции ошибки;
* `_loss` - расчет функции ошибки (для одномерно оптимизации).

## 2. Эксперименты (6 points)

Скачайте датасейт для экспериментов: https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Колонка с ответами - RainTommorow.

In [3]:
import numpy as np

In [ ]:
%load_ext autoreload
%autoreload 2

from ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('weatherAUS.csv')
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')
data.head()

Выделите признаки год/месяц/день:

In [ ]:
data['year'] = # put your code here
data['month'] = # put your code here
data['day'] = # put your code here

Посмотрим какие года есть в выборке:

In [ ]:
data['year'].value_counts()

Разделите выборку на три части (train, val и test) по временному принципу:
    
* train - 2007-2014
* val - 2015
* test - 2016-2017

In [ ]:
indexes = {
    'train': # put your code here,
    'val': # put your code here,
    'test': # put your code here,
}

Здесь вы можете делать всевозможные преобразования признаков. 

Для того, чтобы получить качество, необходимое для преодоления бейзлайна, вам достаточно закодировать все категориальные признаки с помощью `LabelEncoder`, а также разумно обработать пропущенные значения.

In [ ]:
data.drop(['Date'], axis=1, inplace=True)

In [ ]:
#### your code here 

Ваш таргет - **RainTommorow**. Удалите его из обучающих данных, также удалите признак RISK_MM.

In [ ]:
target_data = data['RainTomorrow']
data.drop(['RainTomorrow', 'RISK_MM'], axis=1, inplace=True)

In [ ]:
X_train, y_train = data[indexes['train']].values, target_data[indexes['train']].values
X_val, y_val = data[indexes['val']].values, target_data[indexes['val']].values
X_test, y_test = data[indexes['test']].values, target_data[indexes['test']].values

Для каждого из алгоритмов достигнутое качество (**accuracy**) должно быть: 
* ***RandomForest > 0.84 (2 points)***
* ***GradientBoosting > 0.845 (2 points)***
* ***Отчет об экспериментах (2 points)***

Обучите каждый из алгоритмов до нужного качества, используйте валидационную выборку, чтобы подбирать гиперпараметры. Получите качество (accuracy) выше необходимого и на validation, и на test.

**Подсказка:** для визуализации прогресса обучения можно использовать бибилиотеку [`tqdm`](https://tqdm.github.io/).

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#### your code here

## 3. Бонус. [AdaBoost](https://ru.wikipedia.org/wiki/AdaBoost) (5 points)

В алгоритме AdaBoost всем объектам обучения присваивается вес `weight`, который определяет степень важности объекта при обучении. И если текущая модель ошибается на некотором объекте, его вес увеличивается, и этот объект будет больше влиять на обучение следующей модели. 

Также, так как модели обучаются последовательно, они не равносильны между собой, поэтому у каждой модели тоже есть вес `alpha`, который определяет вес модели при суммировании ответов и зависит от количества ошибок `err` модели. На каждой итерации обучения, эти веса пересчитываются по формулам:

$$\alpha_j = \log\left(\frac{1-err_j}{err_j}\right),$$
где $err_j$ - ошибка классификации

$$w_{new}^t = \frac{w_{old}^{t}*\exp(-\alpha_j \cdot y(x^t) \cdot b_j(x^t))}{\sum\limits_{i=1}^m w_{old}^{t}*\exp(-\alpha_j \cdot y(x^i) \cdot b_j(x^i))}$$

Изначально все веса объектов $w^i$ равны (и нормированы на 1).

Вам предлагается полностью реализовать AdaBoost. Вы можете использовать предыдущие шаблоны, но учтите некоторые пункты:
* надо работать с метками {-1,1} - это обусловлено использованием экспоненциальной функции потерь
* метод `predict` представляет собой функцию сигмоид, примененную к сумме предсказаний всех моделей

Для реализации AdaBoost на датасете, полученном в предыдущем пункте требуется добиться точноcти:
* AdaBoost > 0.83

Полезные ссылки:

- [статья](https://face-rec.org/algorithms/Boosting-Ensemble/decision-theoretic_generalization.pdf)
- [nerc](https://neerc.ifmo.ru/wiki/index.php?title=%D0%91%D1%83%D1%81%D1%82%D0%B8%D0%BD%D0%B3,_AdaBoost)
- [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)